# Symbiotic Relationships Simulation

In [ ]:
import math
import numpy as np
import mesa
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.discrete_space import OrthogonalMooreGrid, CellAgent, FixedAgent, CellCollection
from mesa.experimental.devs import ABMSimulator
import matplotlib.pyplot as plt

SEED = 42

class Animal(CellAgent):
    def __init__(self, model, initial_energy=50, p_reproduce=0.04, energy_from_food=4, mutation_chance=0.5, mutation_effectiveness=0.1, cell=None, symbiotic_property=0.0):
        super().__init__(model)
        self.cell = cell
        self.energy = initial_energy
        self.p_reproduce = p_reproduce
        self.energy_from_food = energy_from_food
        self.symbiotic_property = symbiotic_property
        self.mutation_chance = mutation_chance
        self.mutation_effectiveness = mutation_effectiveness
        
    def feed(self):
        """Abstract method to be implemented by subclasses."""
        
    def step(self):
        self.energy -= 1
        if self.energy <= 0:
            self.remove()
            return

        self.move()
        self.feed()
        
        if self.random.random() < self.p_reproduce:
            self.reproduce()
    
    def move(self):
        self.cell=self.cell.neighborhood.select_random_cell()
        
    def get_symbiotic_property_for_reproduce(self):
        return self.symbiotic_property + self.random.uniform(-self.mutation_effectiveness, self.mutation_effectiveness) if self.random.random() <= self.mutation_chance else self.symbiotic_property
        
    def reproduce(self):
        # """Abstract method to be implemented by subclasses."""
        self.energy /= 2
        self.__class__(
            model = self.model,
            initial_energy = self.energy,
            p_reproduce = self.p_reproduce,
            energy_from_food = self.energy_from_food,
            cell = self.cell,
            symbiotic_property = self.get_symbiotic_property_for_reproduce()
        )


        
class Frog(Animal):
    def feed(self):
        """If possible, eat an ant at current location."""
        ant = [obj for obj in self.cell.agents if isinstance(obj, Ant)]
        if ant:  # If there are any ant present
            ant_to_eat = self.random.choice(ant)
            self.energy += self.energy_from_food
            ant_to_eat.remove()
    
    def move(self):
        """
        Movement rules:
        1. If any neighboring cell (radius=1) has an ant → always move there.
        2. If no ants:
            - Roll chance based on symbiotic_property.
            - If success: find spiders in radius=2.
            - Move ONE STEP toward the nearest spider.
        3. Otherwise: random step to a radius=1 neighbor.
        """

        # 1) Check radius=1 for ants (normal Moore neighborhood) 
        neighbors_r1 = self.cell.get_neighborhood(radius=1).cells
        cells_with_ant = [
            cell for cell in neighbors_r1
            if any(isinstance(obj, Ant) for obj in cell.agents)
        ]

        if cells_with_ant:
            # Always choose one ant cell
            self.cell = self.random.choice(cells_with_ant)
            return


        # 2) No ants -> roll symbiotic chance 
        if self.symbiotic_property > 0:
            if self.random.random() < self.symbiotic_property:

                # radius=2 neighborhood for spider search
                neighbors_r2 = self.cell.get_neighborhood(radius=2).cells

                # Cells containing spiders
                spider_cells = [
                    cell for cell in neighbors_r2
                    if any(isinstance(obj, Spider) for obj in cell.agents)
                ]

                if len(spider_cells) > 0:
                    # Find nearest spider cell
                    fx, fy = self.cell.coordinate

                    def dist(cell):
                        x, y = cell.coordinate
                        return max(abs(x - fx), abs(y - fy))  # Chebyshev distance

                    target_spider_cell = spider_cells[0]
                    sx, sy = target_spider_cell.coordinate

                    # ONE-STEP movement toward spider
                    step_x = fx + (1 if sx > fx else -1 if sx < fx else 0)
                    step_y = fy + (1 if sy > fy else -1 if sy < fy else 0)

                    # Find the cell that corresponds to that coordinate
                    target_neighbor = next(
                        (c for c in neighbors_r1 if c.coordinate == (step_x, step_y)),
                        None
                    )

                    # Move if the step is valid
                    if target_neighbor:
                        self.cell = target_neighbor
                        return
        elif self.symbiotic_property < 0:
            if self.random.random() < (self.symbiotic_property * -1.0):

                # radius=2 neighborhood for spider search
                neighbors_r2 = self.cell.get_neighborhood(radius=2).cells

                # Cells containing spiders
                spider_cells = [
                    cell for cell in neighbors_r2
                    if any(isinstance(obj, Spider) for obj in cell.agents)
                ]

                if len(spider_cells) > 0:
                    # Find nearest spider cell
                    fx, fy = self.cell.coordinate

                    def dist(cell):
                        x, y = cell.coordinate
                        return max(abs(x - fx), abs(y - fy))  # Chebyshev distance

                    target_spider_cell = spider_cells[0]
                    sx, sy = target_spider_cell.coordinate

                    # ONE-STEP movement toward spider
                    step_x = fx + (-1 if sx > fx else 1 if sx < fx else 0)
                    step_y = fy + (-1 if sy > fy else 1 if sy < fy else 0)

                    # Find the cell that corresponds to that coordinate
                    target_neighbor = next(
                        (c for c in neighbors_r1 if c.coordinate == (step_x, step_y)),
                        None
                    )

                    # Move if the step is valid
                    if target_neighbor:
                        self.cell = target_neighbor
                        return


     # --- 3) Default random radius-1 movement ---
        self.cell = self.random.choice(neighbors_r1)

    # def reproduce(self):
        
    #     # self.energy /= 2
    #     self.__class__(
    #         model = self.model,
    #         initial_energy = self.energy,
    #         # self.p_reproduce,
    #         # self.energy_from_food,
    #         cell = self.cell,
    #     )
        

        
class Spider(Animal):
    def __init__(self, model, nest, initial_energy=50, p_reproduce=0.1, energy_from_food=4, cell=None, symbiotic_property = 0.5):
        super().__init__(model=model, 
                         initial_energy=initial_energy, 
                         p_reproduce=p_reproduce, 
                         energy_from_food=energy_from_food, 
                         cell=cell, 
                         symbiotic_property=symbiotic_property)
        self.nest = nest


    def feed(self):
        spider_hit_chance = 0.4
        """If possible, eat a snake at current location."""
        snake = [obj for obj in self.cell.agents if isinstance(obj, Snake)]
        if snake:  # If there are any snake present
            snake_to_eat = self.random.choice(snake)
            self.energy += self.energy_from_food
            snake_to_eat.remove()
            return
        
        ant = [obj for obj in self.cell.agents if isinstance(obj, Ant)]
        if ant and self.random.random() <= spider_hit_chance:  # If there are any ant present
            ant_to_eat = self.random.choice(ant)
            ant_to_eat.remove()
            return
        
        frog = [obj for obj in self.cell.agents if isinstance(obj, Frog)]
        if frog and self.random.random() <= self.symbiotic_property:  # If there are any frog present
            frog_to_eat = self.random.choice(frog)
            self.energy += 1 #might want to change this value later on
            frog_to_eat.remove()
        

    def move(self):
        spider_to_ant_chance = 0.50
        """Move to a neighboring cell, preferably one with frog."""
        cells_with_snake = self.cell.neighborhood.select(
            lambda cell: any(isinstance(obj, Snake) for obj in cell.agents)
        )
        cells_with_ant = self.cell.neighborhood.select(
            lambda cell: any(isinstance(obj, Ant) for obj in cell.agents)
        )
        cells_with_frog = self.cell.neighborhood.select(
            lambda cell: any(isinstance(obj, Frog) for obj in cell.agents)
        )
        explore_factor = 15.0 # Change this value if desired, larger -> more tendency to move away from nest
        target_cells = None
        if len(cells_with_snake) > 0:
            target_cells = cells_with_snake
        elif self.random.random() <= spider_to_ant_chance and len(cells_with_ant) > 0:
            target_cells = cells_with_ant
        elif self.random.random() <= self.symbiotic_property and len(cells_with_frog) > 0:
            target_cells = cells_with_frog
        elif math.dist(self.cell.coordinate, self.get_nest_center()) / explore_factor > self.random.random():
            target_cells = self.determine_cells_to_return()
        else:
            target_cells = self.cell.neighborhood
            
        self.cell = target_cells.select_random_cell()
    
    def get_nest_center(self):
        
        return (self.nest[1][0] + 1, self.nest[1][1] + 1) # This assumes the nest location + 1 is the center of the nest, should be dynamically retrieved based on the nest_size

    def determine_cells_to_return(self):
        destination = self.get_nest_center()
        current_location = self.cell.coordinate
        delta = (destination[0] - current_location[0] , destination[1] - current_location[1]) # Use delta to estimate direction of nest
        
        neighbors_cells = self.cell.neighborhood.cells        
        target_cells = None
        
        # Neighborhood indexing for cursed code:
        # 2 4 7
        # 1 X 6
        # 0 3 5
        if (delta[0] == 0 and delta[1] == 0) or current_location[0] == 0 or current_location[0] == self.model.width-1 or current_location[1] == 0 or current_location[1] == self.model.height-1:  
            # Else we get delta (0, 0) meaning we are located at the nest or if we are at the boundary, just load new neighbors
            target_cells = CellCollection(neighbors_cells)                       
        elif delta[0] < 0 and delta[1] < 0: # LEFT DOWN
            target_cells = CellCollection([neighbors_cells[0], neighbors_cells[1], neighbors_cells[3]])
        elif delta[0] < 0 and delta[1] > 0: # LEFT UP
            target_cells = CellCollection([neighbors_cells[1], neighbors_cells[2], neighbors_cells[4]])
        elif delta[0] > 0 and delta[1] > 0: # RIGHT UP           
            target_cells = CellCollection([neighbors_cells[4], neighbors_cells[7], neighbors_cells[6]])
        elif delta[0] > 0 and delta[1] < 0: # RIGHT DOWN
            target_cells = CellCollection([neighbors_cells[6], neighbors_cells[5], neighbors_cells[3]])
        elif delta[0] == 0 and delta[1] > 0: # UP
            target_cells = CellCollection([neighbors_cells[2], neighbors_cells[4], neighbors_cells[7]])
        elif delta[0] == 0 and delta[1] < 0: # DOWN
            target_cells = CellCollection([neighbors_cells[0], neighbors_cells[5], neighbors_cells[3]])        
        elif delta[0] < 0 and delta[1] == 0: # LEFT
            target_cells = CellCollection([neighbors_cells[0], neighbors_cells[1], neighbors_cells[2]])
        elif delta[0] > 0 and delta[1] == 0: # RIGHT
            target_cells = CellCollection([neighbors_cells[7], neighbors_cells[6], neighbors_cells[5]]) 
        return target_cells
        

    def reproduce(self):
        cells_with_egg = self.cell.neighborhood.select(
        lambda cell: any(isinstance(obj, Egg) for obj in cell.agents)
        )
        if "nest" in self.model.get_zone_at(self.cell.coordinate[0], self.cell.coordinate[1]) and self.model.get_zone_at(self.cell.coordinate[0], self.cell.coordinate[1]) not in cells_with_egg and len(cells_with_egg) < 16:
            Egg.create_agents(
                self.model,
                1,
                cell=self.cell,
                nest = self.nest,
                symbiotic_property = self.get_symbiotic_property_for_reproduce()
                )

class Ant(Animal):
    def move(self):
        cells_with_egg = self.cell.neighborhood.select(
            lambda cell: any(isinstance(obj, Egg) for obj in cell.agents)
        )
        target_cells = (
            cells_with_egg if len(cells_with_egg) > 0 else self.cell.neighborhood
        )
        self.cell = target_cells.select_random_cell()
    
    

class Snake(Animal):
    def feed(self):
        """If possible, eat a frog at current location."""
        frog = [obj for obj in self.cell.agents if isinstance(obj, Frog)]
        if frog:  # If there are any frog present
            frog_to_eat = self.random.choice(frog)
            self.energy += self.energy_from_food
            frog_to_eat.remove()

    def move(self):
        """Move to a neighboring cell, preferably one with frog."""
        cells_with_sheep = self.cell.neighborhood.select(
            lambda cell: any(isinstance(obj, Frog) for obj in cell.agents)
        )
        target_cells = (
            cells_with_sheep if len(cells_with_sheep) > 0 else self.cell.neighborhood
        )
        self.cell = target_cells.select_random_cell()

class Egg(CellAgent):
    def __init__(self, model, nest,symbiotic_property, cell=None , hp = 5):
        super().__init__(model)
        self.cell = cell
        self.hp = hp
        self.egg_placement_step = self.model.steps
        self.nest = nest
        self.symbiotic_property = symbiotic_property

    def step(self):
        if self.is_ant_nearby():
            self.hp -= 1
        if self.hp <= 0:
            self.remove() 
        if self.model.steps - self.egg_placement_step >= 5 and self.hp > 0:
            self.hatch()
            self.remove()
            pass

    def is_ant_nearby(self):
        nearby_ants = self.cell.neighborhood.select(
            lambda cell: any(isinstance(obj, Ant) for obj in cell.agents))
        return len(nearby_ants) > 0

    def hatch(self):
        # nest_key = self.model.get_zone_at(self.cell.coordinate[0], self.cell.coordinate[1])
        # nest = self.model.zones.get(nest_key)
        Spider.create_agents(
                self.model,
                1, # Ant amount
                cell=self.cell,
                symbiotic_property = self.symbiotic_property,
                nest =self.nest)

        

    

In [ ]:
class SymbioticRelationshipsModel(Model):
    def __init__(self,
                 grid_size=32,
                 initial_frogs=10,
                 #initial_spiders=10,
                 initial_ants=10,
                 initial_snakes=10,
                 mutation_rate=0.5,
                 nest_density=0.75,
                 seed = None,
                 rng = None,
                 simulator: ABMSimulator = None):
        super().__init__(seed=seed, rng=rng)
        
        if simulator is None:
            simulator = ABMSimulator()

        self.simulator = simulator
        self.simulator.setup(self)
        
        self.height = grid_size
        self.width = grid_size
        nest_density = 1-nest_density  
        # Create grid using experimental cell space
        self.grid = OrthogonalMooreGrid(
            [self.height, self.width],
            torus=False, # Decide upon this!?
            capacity=math.inf, #
            random=self.random,
        )
        
        # Create nest zone mapping for tracking
        self.zones = {}
        
        self.spider_nests = {}
        self.spider_nest_size = 3

        margin = self.spider_nest_size     # distance from walls to keep free
        x0 = margin
        y0 = margin
        x1 = self.width - margin
        y1 = self.height - margin
        dx = int(self.width * nest_density)
        dy = int(self.height * nest_density)

        nest_count = 0

        for x in range(x0, x1, dx):
            for y in range(y0, y1, dy):
                nest_count += 1

                nx = x - self.spider_nest_size // 2
                ny = y - self.spider_nest_size // 2
                self.spider_nests[f"nest{nest_count}"] = (nx, ny)
            
        # self.spider_nests = {
        #               "nest1": (4, 4),
        #               "nest2": (4, 26),
        #               "nest3": (26, 4),
        #               "nest4": (26, 26),
        # }

        for cell in self.grid.all_cells.cells:
            x, y = cell.coordinate
            
            # Mark spider nests
            for nest_name, nest_location in self.spider_nests.items():
                start_nest_x = nest_location[0]
                end_nest_x = nest_location[0] + self.spider_nest_size
                start_nest_y = nest_location[1]
                end_nest_y = nest_location[1] + self.spider_nest_size
                if x >= start_nest_x and x< end_nest_x and y >= start_nest_y and y < end_nest_y:
                    self.zones[(x, y)] = nest_name
        
        # Spawn spiders on their nests
        for nest_name, nest_location in self.spider_nests.items():
            Spider.create_agents(
                                self,
                                1, # Spider amount
                                nest = (nest_name, nest_location),
                                cell=self.random.choices([cell for cell in self.grid.all_cells.cells if cell.coordinate[0] == nest_location[0] + 1 and cell.coordinate[1] == nest_location[1] + 1], k=1)) # This can't be right btw,
                    
                            
        # Set up data collection
        model_reporters = {
            "Spiders": lambda m: len(m.agents_by_type[Spider]),
            "Frogs": lambda m: len(m.agents_by_type[Frog]),
            "Ants": lambda m: len(m.agents_by_type[Ant]),
            "Snakes": lambda m: len(m.agents_by_type[Snake]),
            "Spider_Symb_Val": lambda m: np.mean(list(map(lambda spider: spider.symbiotic_property, m.agents_by_type[Spider]))),
            "Frog_Symb_Val": lambda m: np.mean(list(map(lambda frog: frog.symbiotic_property, m.agents_by_type[Frog])))
        }

        self.datacollector = DataCollector(model_reporters)
        
        
        Frog.create_agents(
            self,
            initial_frogs, # frog amount
            cell=self.random.choices(self.grid.all_cells.cells, k=initial_frogs),symbiotic_property=0.8,
        )

        Snake.create_agents(
            self,
            initial_snakes, # Snake amount
            cell=self.random.choices(self.grid.all_cells.cells, k=initial_snakes),
        )

        Ant.create_agents(
            self,
            initial_ants, # Ant amount
            cell=self.random.choices(self.grid.all_cells.cells, k=initial_ants),
        )
        
        # Egg.create_agents(
        #     self,
        #     1, # Ant amount
        #     cell=self.random.choices(self.grid.all_cells.cells, k=1),
        # )
        
        # Collect initial data
        self.running = True
        self.datacollector.collect(self)
        
    def get_zone_at(self, x, y):
        return self.zones.get((x, y), "unmarked")
    
    def step(self):
        """Execute one step of the model."""
        self.agents_by_type[Ant].shuffle_do("step")
        self.agents_by_type[Snake].shuffle_do("step")
        self.agents_by_type[Frog].shuffle_do("step")
        self.agents_by_type[Spider].shuffle_do("step")
        try:
            self.agents_by_type[Egg].shuffle_do('step')
        except:
            pass
        
        # Collect data
        self.datacollector.collect(self)
        
        # Spawn ants every 2 ticks
        if self.steps % 2 == 0:
            Ant.create_agents(
                self,
                1, # Ant amount
                cell=self.grid.select_random_empty_cell()) # Might spawn at unlucky place (against nest), will that be a problem?!?!
            
        



In [ ]:
# use Rectangle to draw background zones using patches
from matplotlib.patches import Rectangle
import solara

@solara.component
def CustomSpaceVisualization(model):
    """Custom space visualization with colored background zones"""
    # This is required to update the visualization when the model steps
    from mesa.visualization.components.matplotlib_components import update_counter
    update_counter.get()  # This triggers re-rendering on model updates
    # Setup figure
    fig = plt.Figure(figsize=(8, 8))    # Note: you must initialize a figure using this method instead of
    ax = fig.subplots()
    # set the width and height based on model
    width, height = model.width, model.height
    # Draw home area (light blue background)

    
   
    # Define nest areas
    for nest_location in model.spider_nests.values():
        start_nest_x = nest_location[0]
        start_nest_y = nest_location[1]
        nest = Rectangle((start_nest_x, start_nest_y), model.spider_nest_size, model.spider_nest_size,
                linewidth=0, facecolor='lightcoral', alpha=0.2)
        ax.add_patch(nest)
            # self.barcells.append(cell)
            
    # Draw our agents
    for frog in model.agents_by_type[Frog]:
        x, y = frog.cell.coordinate
        
        color = "tab:green"
        marker = "o"
        size = 100
        #     # Use matplotlib's OOP API instead of plt.scatter for thread safety
        ax.scatter(x + 0.5, y + 0.5, c=color, s=size, marker=marker, zorder=10, alpha=0.8)
        
    for spider in model.agents_by_type[Spider]:
        x, y = spider.cell.coordinate
        
        color = "tab:brown"
        marker = "X"
        size = 100
        #     # Use matplotlib's OOP API instead of plt.scatter for thread safety
        ax.scatter(x + 0.5, y + 0.5, c=color, marker=marker, s=size, zorder=10, alpha=0.8)
        
    for ant in model.agents_by_type[Ant]:
        x, y = ant.cell.coordinate
        
        color = "tab:red"
        marker = "d"
        size = 100
        #     # Use matplotlib's OOP API instead of plt.scatter for thread safety
        ax.scatter(x + 0.5, y + 0.5, c=color, marker=marker, s=size, zorder=10, alpha=0.8)
    
    for snake in model.agents_by_type[Snake]:
        x, y = snake.cell.coordinate
            
        color = "tab:orange"
        marker = "v"
        size = 100
        ax.scatter(x + 0.5, y + 0.5, c=color, marker=marker, s=size, zorder=10, alpha=0.8)

    try:
        for egg in model.agents_by_type[Egg]:
            x, y = egg.cell.coordinate
            
            color = "tab:blue"
            marker = "s"
            size = 100
            ax.scatter(x + 0.5, y + 0.5, c=color, marker=marker, s=size, zorder=10, alpha=0.8)
    except:
        pass

                
    # Set up the plot
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('X coordinate')
    ax.set_ylabel('Y coordinate')
    
    # This is required to render the visualization
    solara.FigureMatplotlib(fig)

In [ ]:
from mesa.visualization import (
    CommandConsole,
    Slider,
    SolaraViz,
    make_plot_component,
    make_space_component,
)

# This function will be called for every agent present in the model and defines a visualization (color, marker...)
def agent_portrayal(agent):
    if agent is None:
        return

    portrayal = {
        # "size": 25,
    }


    # Filter the agent type, then describe the agent visuals
    if isinstance(agent, Frog):
        portrayal["color"] = "tab:green"
        portrayal["marker"] = "o"
        portrayal["zorder"] = 2

    elif isinstance(agent, Spider):
        portrayal["color"] = "tab:brown"
        portrayal["marker"] = "X"
        portrayal["zorder"] = 2

    elif isinstance(agent, Ant):
        portrayal["color"] = "tab:red"
        portrayal["marker"] = "d"
        portrayal["zorder"] = 2

    elif isinstance(agent, Snake):
        portrayal["color"] = "tab:orange"
        portrayal["marker"] = "s"
        portrayal["zorder"] = 2
    

    return portrayal


model_params = {
    "seed": {
        "type": "InputText",
        "value": SEED,
        "label": "Random Seed",
    },
    "initial_frogs": Slider("Initial Frog Population", 10, 1, 200),
    'initial_snakes': Slider("Initial Snake Population",10, 1, 200),
    "initial_ants" : Slider("Initial Ant Population",10, 1, 200),
    "grid_size" : Slider("Size of grid", 32, 32, 256, 4),
    "nest_density" : Slider("Nest density", 0.20, 0.1, 1, 0.05),
}

def post_process_space(ax):
    ax.set_aspect("equal")
    ax.set_xticks([])
    ax.set_yticks([])

# The main grid with all the agents
space_component = make_space_component(
    agent_portrayal, draw_grid=False, post_process=post_process_space
)

def post_process_lines(ax):
    ax.legend(loc="center left", bbox_to_anchor=(1, 0.9))

simulator = ABMSimulator() 
model = SymbioticRelationshipsModel(seed=SEED, initial_ants=10, initial_frogs=10, initial_snakes=10, nest_density=0.20, simulator=simulator)

lineplot_component = make_plot_component(
    {"Frogs": "tab:green", "Spiders": "tab:brown","Snakes": "tab:orange","Ants":"tab:red"},
    post_process=post_process_lines,
)

lineplot_component2 = make_plot_component(
    {"Frog_Symb_Val": "tab:green", "Spider_Symb_Val": "tab:brown"},
    post_process=post_process_lines,
)

page = SolaraViz(
    model,
    # components=[space_component, lineplot_component, CommandConsole],
    components=[CustomSpaceVisualization, lineplot_component, lineplot_component2, CommandConsole],
    model_params=model_params,
    name="Symbiotic Relationships",
    simulator=simulator,
)

page  # noqa